## Question 7.5
### Sorting algorithm via MapReduce

Load file from bucket as a RDD object:

In [185]:
applicants = sc.textFile('s3://admhw3/ApplicantsInfo.txt')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Count number of elements (rows):

In [186]:
applicants.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

50001

Check and save dimensions of `applicants`:

In [187]:
#Check dimensionality: n students, m exam grades
applicants.take(1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['50000 1000']

In [188]:
#Memorize n and m
n_students = int(applicants.take(1)[0].split(' ')[0])
n_grades = int(applicants.take(1)[0].split(' ')[1])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Get rid of the file header, as it only contains the number of rows and columns:

In [189]:
firstline = applicants.first()
applicants = applicants.filter(lambda row: row != firstline)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### **First solution**
This solution seems to work, but doesn't involve any `reduce` operations:

In [195]:
applicants.map(lambda row: (' '.join(row.split(' ')[:2]), [int(x) for x in row.split(' ')[2:]]))\
.map(lambda t: (t[0], sum(t[1])/n_grades))\
.takeOrdered(5, key = lambda t: -t[1])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[('Emily Crispin', 24.492), ('Patricia Witten', 24.489), ('Doreen Richmond', 24.447), ('Bruce Johnson', 24.445), ('David Niederberger', 24.437)]

### **Second solution**
To force the use of `reduce`, we reshape the RDD object in a list of tuples (name_of_student, grade), reduce them by summing all the grades of the same student, finally divide by the number of grades (that is fixed) and return first 5 students in order of average grades:

In [203]:
applicants.map(lambda row: (' '.join(row.split(' ')[:2]), [int(x) for x in row.split(' ')[2:]]))\
.flatMap(lambda t: [(t[0], t[1][i]) for i in t[1]])\
.reduceByKey(lambda a, b: a + b)\
.map(lambda t: (t[0], t[1]/n_grades))\
.takeOrdered(5, key = lambda t: -t[1])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[('William Williams', 287.902), ('Michael Smith', 283.971), ('James Johnson', 244.221), ('Michael Martin', 241.382), ('Robert Johnson', 239.785)]

Of course, the above values are not what we expected to see.

We have the doubt that the file contains duplicated names of students, so we check how many unique names are there:

In [204]:
applicants.map(lambda row: (' '.join(row.split(' ')[:2]), [int(x) for x in row.split(' ')[2:]]))\
.groupBy(lambda x: x[0]).distinct().count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

47581

If this is the case, it's a problem: in fact the function previously provided will sum together the grades of an unknown number of different people that share the same name. We thought about some possible solutions involving zipWithIndex(), but didn't manage to make it work. Below we report one of these failed attempts just for the record, as it didn't return any meaningful result.

In [214]:
applicants.map(lambda row: (' '.join(row.split(' ')[:2]), [int(x) for x in row.split(' ')[2:]]))\
.flatMap(lambda t: [(t[0], t[1][i]) for i in t[1]]).zipWithIndex().map(lambda x: (x[1], x[0]))\
.reduceByKey(lambda a, b: a[1] + b[1])\
.map(lambda t: (t[0], (t[1][0], t[1][1]/n_grades)))\
.takeOrdered(5, key = lambda t: -t[1][1])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[(22320, ('Debra Amato', 0.03)), (222375, ('Leah Fields', 0.03)), (225843, ('John Reilly', 0.03)), (246780, ('Carmen Smith', 0.03)), (1007187, ('William Steege', 0.03))]